<a href="https://colab.research.google.com/github/AbhijithX0/2025-Frontend-dev-with-react-ADDON-Abhijith-binu-cse-ai-ml/blob/main/ML_Project_military_object_identification_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install ultralytics

In [31]:
from google.colab import files

# This command opens a file selection dialog.
# Please select your kaggle.json file when prompted.
uploaded = files.upload()

# The file(s) are now in the current working directory, e.g., '/content/'.
# The 'uploaded' dictionary contains the file name as a key.
for filename in uploaded.keys():
    print(f"File '{filename}' uploaded successfully!")

Saving kaggle (1).json to kaggle (1).json
File 'kaggle (1).json' uploaded successfully!


In [32]:
!pip install ultralytics
from ultralytics import YOLO
import os

# Load your best trained model weights.
# The path below assumes your last training run was named 'train'.
# If you have multiple runs, you may need to update 'train' to 'train2', 'train3', etc.
model_path = "/content/runs/detect/train8/weights/best.pt"

# Check if the model file exists
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
else:
    model = YOLO(model_path)

    # Get the name of the uploaded file.
    # We'll assume you only uploaded one file for this example.
    uploaded_filename = list(uploaded.keys())[0]

    # Construct the full path to the uploaded image.
    image_path = f"/content/{uploaded_filename}"

    # Run inference on the single, uploaded image.
    # The results will be stored in the 'results' object.
    results = model(image_path)

    # Save and display the results.
    # The r.save() method will save the predicted image with bounding boxes
    # and labels to a new directory like 'runs/detect/predict'.
    for r in results:
        r.save()
        r.show()

Error: Model file not found at /content/runs/detect/train8/weights/best.pt


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d sudipchakrabarty/kiit-mita


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [ ]:
!unzip kiit-mita.zip


unzip:  cannot find or open kiit-mita.zip, kiit-mita.zip.zip or kiit-mita.zip.ZIP.


In [ ]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolo11n.pt")

In [ ]:
import os, cv2, yaml, random, numpy as np
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt
from torchvision import transforms as T

class Visualization:

    def __init__(self, root, data_types, n_ims, rows, cmap=None):
        self.root = root   # ✅ FIX: Save root
        self.n_ims, self.rows = n_ims, rows
        self.cmap, self.data_types = cmap, data_types
        self.colors = ["firebrick", "darkorange", "blueviolet"]

        self.get_cls_names()
        self.get_bboxes()

    def get_cls_names(self):
        with open(f"{self.root}/KIIT-MiTA.yml", 'r') as file:   # ✅ FIX: use self.root
            data = yaml.safe_load(file)
        # Extract class names
        class_names = data['names']

        # Create a dictionary with index as key and class name as value
        self.class_dict = {index: name for index, name in enumerate(class_names)}

    def get_bboxes(self):
        self.vis_datas, self.analysis_datas, self.im_paths = {}, {}, {}
        for data_type in self.data_types:
            all_bboxes, all_analysis_datas = [], {}
            im_paths = glob(f"{self.root}/{data_type}/images/*")   # ✅ FIX: use self.root

            for idx, im_path in enumerate(im_paths):
                bboxes = []
                im_ext = os.path.splitext(im_path)[-1]
                lbl_path = im_path.replace(im_ext, ".txt")
                lbl_path  = lbl_path.replace(f"{data_type}/images/", f"{data_type}/labels/")
                if not os.path.isfile(lbl_path): continue
                meta_data = open(lbl_path).readlines()
                for data in meta_data:
                    parts = data.strip().split()[:5]
                    cls_name = self.class_dict[int(parts[0])]
                    bboxes.append([cls_name] + [float(x) for x in parts[1:]])
                    if cls_name not in all_analysis_datas:
                        all_analysis_datas[cls_name] = 1
                    else:
                        all_analysis_datas[cls_name] += 1
                all_bboxes.append(bboxes)

            self.vis_datas[data_type] = all_bboxes
            self.analysis_datas[data_type] = all_analysis_datas
            self.im_paths[data_type] = im_paths

    def plot(self, rows, cols, count, im_path, bboxes):
        plt.subplot(rows, cols, count)
        or_im = np.array(Image.open(im_path).convert("RGB"))
        height, width, _ = or_im.shape

        for bbox in bboxes:
            class_id, x_center, y_center, w, h = bbox
            # Convert YOLO format to pixel values
            x_min = int((x_center - w / 2) * width)
            y_min = int((y_center - h / 2) * height)
            x_max = int((x_center + w / 2) * width)
            y_max = int((y_center + h / 2) * height)

            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            cv2.rectangle(img=or_im, pt1=(x_min, y_min), pt2=(x_max, y_max), color=color, thickness=3)
        plt.imshow(or_im)
        plt.axis("off")
        plt.title(f"There is (are) {len(bboxes)} object(s) in the image.")

        return count + 1

    def vis(self, save_name):
        print(f"{save_name.upper()} Data Visualization is in process...\n")
        assert self.cmap in ["rgb", "gray"], "Please choose rgb or gray cmap"
        if self.cmap == "rgb": cmap = "viridis"
        cols = self.n_ims // self.rows
        count = 1

        plt.figure(figsize=(25, 20))

        indices = [random.randint(a=0, b=len(self.vis_datas[save_name]) - 1) for _ in range(self.n_ims)]

        for idx, index in enumerate(indices):
            if count == self.n_ims + 1: break
            im_path, bboxes = self.im_paths[save_name][index], self.vis_datas[save_name][index]
            count = self.plot(self.rows, cols, count, im_path=im_path, bboxes=bboxes)

        plt.show()

    def data_analysis(self, save_name, color):
        print("Data analysis is in process...\n")

        width, text_width, text_height = 0.7, 0.05, 2
        cls_names = list(self.analysis_datas[save_name].keys())
        counts = list(self.analysis_datas[save_name].values())

        _, ax = plt.subplots(figsize=(30, 10))
        indices = np.arange(len(counts))

        ax.bar(indices, counts, width, color=color)
        ax.set_xlabel("Class Names", color="black")
        ax.set(xticks=indices, xticklabels=cls_names)   # ✅ FIX for warning
        ax.set_ylabel("Data Counts", color="black")
        ax.set_title(f"{save_name.upper()} Dataset Class Imbalance Analysis")

        for i, v in enumerate(counts):
            ax.text(i - text_width, v + text_height, str(v), color="royalblue")

    def visualization(self):
        [self.vis(save_name) for save_name in self.data_types]

    def analysis(self):
        [self.data_analysis(save_name, color) for (save_name, color) in zip(self.data_types, self.colors)]


# ✅ Example usage
root = "/content/KIIT-MiTA"

vis = Visualization(root=root, data_types=["train", "valid", "test"], n_ims=20, rows=5, cmap="rgb")
vis.analysis()
vis.visualization()


FileNotFoundError: [Errno 2] No such file or directory: '/content/KIIT-MiTA/KIIT-MiTA.yml'

In [ ]:
import os, yaml

# ✅ Add this line to define the 'root' variable
root = "/content/KIIT-MiTA"

yml_file_path = f"{root}/KIIT-MiTA.yml"

# Load original dataset YAML
with open(yml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Update paths for Kaggle
# Correct paths (absolute)
data['train'] = '/content/KIIT-MiTA/train/images'
data['val']   = '/content/KIIT-MiTA/valid/images'
data['test']  = '/content/KIIT-MiTA/test/images'

print("Updated YAML Config:\n", data)

# ✅ Ensure /kaggle/working exists before saving
os.makedirs("/kaggle/working", exist_ok=True)

# Save updated YAML
output_path = '/kaggle/working/updated_config.yml'
with open(output_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"✅ YAML config saved at: {output_path}")

# 🔥 Train YOLO model
train_results = model.train(
    data=output_path,    # updated YAML
    epochs=10,           # training epochs
    imgsz=480,           # image size
    device=0             # use single GPU
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/KIIT-MiTA/KIIT-MiTA.yml'

In [ ]:
from PIL import Image
from IPython.display import display

# Open the image file
image = Image.open("/content/runs/detect/train8/confusion_matrix.png")

# Display the image in the notebook
display(image)

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/confusion_matrix.png'

In [ ]:
Image.open(f"/content/runs/detect/train8/results.png")

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/results.png'

In [ ]:
Image.open(f"/content/runs/detect/train8/BoxP_curve.png")

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/BoxP_curve.png'

In [ ]:
Image.open(f"/content/runs/detect/train8/BoxR_curve.png")

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/BoxR_curve.png'

In [ ]:
Image.open(f"/content/runs/detect/train8/BoxPR_curve.png")

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/BoxPR_curve.png'

In [ ]:
Image.open(f"/content/runs/detect/train8/BoxF1_curve.png")

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/BoxF1_curve.png'

In [ ]:
Image.open(f"/content/runs/detect/train8/val_batch0_pred.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train8/val_batch0_pred.jpg'

In [ ]:
from ultralytics import YOLO
import os

# Load your best trained model weights from the correct directory.
# You may need to update 'train' to 'train2', 'train3', etc.
model = YOLO("/content/runs/detect/train8/weights/best.pt")

# Define the path to the test images folder
test_images_path = "/content/KIIT-MiTA/test/images"

# Run inference on all images in the folder.
# 'device=0' uses the first GPU. 'verbose=False' hides the per-image output.
inference_results = model(test_images_path, device=0, verbose=False)

# Loop through the results to access each image's predictions and save them.
# The results are saved to 'runs/detect/predict' by default.
for r in inference_results:
    # r.save() will save the predicted image with bounding boxes.
    # The default location is a new directory under runs/detect/
    r.save()

print("Inference complete. Predicted images are saved in a new runs/detect/predict directory.")

Inference complete. Predicted images are saved in a new runs/detect/predict directory.


In [ ]:
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

def inference_vis(res, n_ims, rows, class_names):
    """
    Visualizes inference results by displaying images with bounding boxes,
    class labels, and confidence scores.

    Args:
        res (list): List of Ultralytics Results objects.
        n_ims (int): The total number of images to display.
        rows (int): The number of rows in the plot grid.
        class_names (list): A list of class names from the dataset YAML.
    """
    cols = n_ims // rows
    plt.figure(figsize=(20, 10))
    for idx, r in enumerate(res):
        if idx >= n_ims:
            break

        plt.subplot(rows, cols, idx + 1)
        or_im_rgb = np.array(Image.open(r.path).convert("RGB"))

        # Iterate through each bounding box
        for bbox in r.boxes:
            box = bbox.xyxy[0]
            x1, y1, x2, y2 = box.cpu().numpy().astype(int)

            # Get class ID and confidence score
            class_id = int(bbox.cls)
            conf_score = float(bbox.conf)
            class_name = class_names[class_id]

            # Format the label string
            label = f"{class_name}: {conf_score:.2f}"

            # Draw the bounding box
            cv2.rectangle(or_im_rgb, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)

            # Draw the label text
            cv2.putText(
                or_im_rgb,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
                2,
            )

        plt.imshow(or_im_rgb)
        plt.title(f"Image #{idx + 1}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# You must define your class names first, from your YAML file.
class_names = ['Artillary', 'Missile', 'Radar', 'M. Rocket Launcher', 'Soldier', 'Tank', 'Vehicle']

# Then call the function with the new class_names parameter
inference_vis(res=inference_results, n_ims=15, rows=3, class_names=class_names)

NameError: name 'inference_results' is not defined

In [ ]:
from google.colab import files
files.upload()

Saving images.jpg to images.jpg


{'images.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x12\x13\x12\x15\x12\x12\x13\x15\x15\x15\x16\x17\x15\x18\x15\x17\x18\x17\x18\x15\x1a\x18\x17\x16\x17\x18\x16\x17\x17\x17\x18\x18\x1d( \x18\x1a%\x1f\x15\x16!1!%)-...\x18\x1f383-7(-.+\x01\n\n\n\x0e\r\x0e\x1b\x10\x10\x1a-\x1d %-----/----------+---------------+----------+----77\xff\xc0\x00\x11\x08\x00\xb1\x01\x1d\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x00\x07\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x00\x08\xff\xc4\x00@\x10\x00\x02\x01\x02\x04\x03\x06\x04\x04\x04\x05\x04\x01\x05\x00\x00\x01\x02\x11\x00\x03\x04\x12!1\x05AQ\x06\x13"aq\x81\x072\x91\xb1B\xa1\xc1\xf0\x14#R\xd13br\x82\xe1\x15\x92\xa2\xf1C$4S\xb2\xc2\xff\xc4\x00\x19\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x02\x03\x04\x05\xff\xc4\x00#\x11\x01\x01\x00\x02\x02\x02\x03\x00\x02\x03\x00\x00\x00\x00\x00\x00\x00\x01\x02\x11!1\x12